In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import json
import re

In [2]:
# Đường dẫn đến WebDriver (đổi tùy theo trình duyệt bạn sử dụng)
webdriver_path = 'D:/Code/Python/f1_crawlling/chromedriver_win32/chromedriver.exe'

In [3]:

# Tạo các tùy chọn cho trình duyệt
chrome_options = Options()
chrome_options.add_argument('--headless')  # Chạy trình duyệt ẩn danh (không hiển thị trình duyệt)
chrome_options.binary_location = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe" # Đường dẫn đến trình duyệt Chrome

# Khởi tạo trình duyệt (ở đây sử dụng Chrome)
driver = webdriver.Chrome(options=chrome_options)


url = 'https://www.dienmayxanh.com/may-giat/casper-inverter-8-kg-wf-8vg1'


driver.get(url)



In [4]:
# Lấy thông tin từ phần view-source
soup = BeautifulSoup(driver.page_source, 'html.parser')
title = soup.find('h3', class_='article__content__title').text.strip()

# Lấy thông tin từ đoạn script JSON
product_ld_script = soup.find('script', {'id': 'productld'})
product_ld_json = json.loads(product_ld_script.contents[0])

# Lấy thông tin từ div chứa mô tả sản phẩm
product_description_div = soup.find('div', class_='content-article')
product_description = product_description_div.find_all(['p', 'h3'])  # Tìm tất cả các thẻ p và h3 trong product_description_div

In [5]:
# Tạo danh sách chứa nội dung của từng thẻ p và h3
content_list = []
for tag in product_description:
    # Kiểm tra xem tag có phải là thẻ a không
    if tag.find('a', class_='preventdefault'):
        break  # Nếu là thẻ a, dừng vòng lặp
    content_list.append(tag.get_text(strip=True, separator='\n'))

# Ghép các nội dung trong danh sách thành một chuỗi
product_description_text = '\n'.join(content_list)

# Tạo dictionary chứa thông tin cần lấy
product_details = {
    'title': title,
    'name': product_ld_json.get('name', ''),
    'description': product_ld_json.get('description', ''),
    'sku': product_ld_json.get('sku', ''),
    'brand': product_ld_json.get('brand', {}).get('name', ''),
    'product_description': product_description_text,
    # Thêm các thông tin khác tùy theo nhu cầu
}

In [6]:
# Lấy thông tin từ đoạn script JSON khác
additional_script = soup.find('script', {'type': 'application/ld+json', 'id': 'productld'})
additional_script_json = json.loads(additional_script.contents[0])

# Thêm thông tin từ đoạn script JSON khác vào product_details
product_details.update(additional_script_json)

# Chuyển dictionary thành chuỗi JSON và in ra
result_json = json.dumps({'product_details': product_details}, ensure_ascii=False, indent=4)
print(result_json)

# Đóng trình duyệt
driver.quit()

{
    "product_details": {
        "title": "Thông tin sản phẩm",
        "name": "Máy giặt Casper Inverter 8 kg WF-8VG1",
        "description": "Máy giặt Máy giặt Casper Inverter 8 kg WF-8VG1 giá rẻ chính hãng, trả góp 0%, bảo hành chính hãng - Giao hàng tận nơi, lắp đặt nhanh chóng, khuyến mãi hấp dẫn tại Điện máy XANH!",
        "sku": "316033",
        "brand": {
            "@type": "Brand",
            "name": [
                "Casper"
            ]
        },
        "product_description": "Máy giặt Casper Inverter 8 kg WF-8VG1 trang bị 16 chương trình giặt đa dạng hỗ trợ đắc lực nhu cầu giặt giũ trong gia đình với công nghệ giặt hơi nước Steam Wash diệt khuẩn 99.99%, công nghệ suy luận ảo Fuzzy Logic xác định khối lượng quần áo giúp tiết kiệm nước, công nghệ Inverter tiết kiệm điện năng.\nTổng quan thiết kế\n- Kiểu dáng:\nmáy giặt cửa trước\nthiết kế tối giản và sang trọng với gam màu tối thanh lịch, phù hợp với nhiều không gian nội thất khác nhau.\n-\nBảng điều khiển tiếng V